In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'M40'
file_name = 'M40'

In [3]:
DATA_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/")

## Dense Layer

In [4]:
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/2/benchmark_dense__20181119.pkl' %file_name))

for i in range(3,3):
    dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20181119.pkl' %(file_name, i)))])

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,32.784080,2037.468080,2042.244520,0.0,32.0,0.732888,0.701663,0.679047,0.019520,1.365308e+08,...,0.083280,0.083080,0.084800,0.493120,0.085440,0.085040,0.251960,0.249560,0.251560,0.246920
std,18.460976,1187.081709,1183.780913,0.0,0.0,0.395866,0.391203,0.392509,0.009891,1.597323e+08,...,0.276311,0.276009,0.278589,0.499963,0.279541,0.278947,0.434147,0.432767,0.433918,0.431228
min,1.000000,1.000000,1.000000,0.0,32.0,0.346479,0.326743,0.274577,0.000976,7.600000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,996.000000,1011.000000,0.0,32.0,0.456280,0.428758,0.402660,0.012755,2.204244e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,2043.000000,2035.000000,0.0,32.0,0.592036,0.558889,0.538108,0.017729,7.419462e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,3065.250000,3077.250000,0.0,32.0,0.845661,0.812736,0.792259,0.024069,1.949631e+08,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,64.000000,4095.000000,4095.000000,0.0,32.0,3.094482,3.024440,2.880960,0.117104,1.031366e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [5]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,activation_fct,use_bias,optimizer,gpu,timeUsed_median,timeUsed_min,timeUsed_max,timeUsed_std
0,39,233,2,31,28,4,0,32,3,False,3,0,21.948586,21.839061,21.997762,0.060845
1,11,33,4,58,22,2,1,32,2,False,6,0,1.290812,1.222858,1.315994,0.032490
2,16,468,1,16,5,4,1,32,0,True,1,0,11.852250,11.786218,11.892452,0.038696
3,14,56,7,91,48,1,1,32,1,True,0,0,3.187203,3.136878,3.233252,0.037373
4,2,396,2,16,11,4,0,32,2,True,0,0,0.598645,0.584321,0.615149,0.009852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,52,509,3,11,17,1,1,32,1,True,0,0,62.420244,62.364659,62.477679,0.041832
2496,39,83,6,69,16,4,0,32,2,False,1,0,8.040805,7.943897,8.050637,0.048160
2497,11,433,7,19,9,2,0,32,3,True,3,0,42.953763,42.857685,43.074341,0.076472
2498,14,308,5,25,30,1,1,32,3,False,0,0,16.456604,16.367044,16.482391,0.046634


In [6]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [7]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [8]:
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
0,39,233,2,31,28,4,0,32,0,0,...,1,0,0,0,0,0,0,0,1,0
1,11,33,4,58,22,2,1,32,0,0,...,0,0,0,0,1,0,0,0,0,1
2,16,468,1,16,5,4,1,32,1,0,...,0,0,0,0,0,1,1,0,0,0
3,14,56,7,91,48,1,1,32,1,0,...,0,0,0,1,0,0,0,1,0,0
4,2,396,2,16,11,4,0,32,1,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,52,509,3,11,17,1,1,32,1,0,...,0,0,0,1,0,0,0,1,0,0
2496,39,83,6,69,16,4,0,32,0,0,...,0,0,0,0,0,1,0,0,0,1
2497,11,433,7,19,9,2,0,32,1,0,...,1,0,0,0,0,0,0,0,1,0
2498,14,308,5,25,30,1,1,32,0,0,...,0,0,0,1,0,0,0,0,1,0


In [9]:
dfConv.describe()

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.0,2497.000000,2497.0,...,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000
mean,32.722467,260.261514,4.024429,61.382058,59.101322,2.500601,0.497797,32.0,0.484982,0.0,...,0.086504,0.093312,0.085302,0.493793,0.081698,0.088907,0.254706,0.247097,0.246296,0.251902
std,18.587250,149.087352,2.007748,308.436623,289.606259,1.105466,0.500095,0.0,0.499875,0.0,...,0.281163,0.290927,0.279387,0.500062,0.273959,0.284666,0.435783,0.431410,0.430939,0.434192
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,32.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,128.000000,2.000000,9.000000,10.000000,2.000000,0.000000,32.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,256.000000,4.000000,19.000000,19.000000,2.000000,0.000000,32.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,393.000000,6.000000,38.000000,37.000000,3.000000,1.000000,32.0,1.000000,0.0,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,64.000000,512.000000,7.000000,9868.000000,9083.000000,4.000000,1.000000,32.0,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))